In [193]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flight-fare-prediction-mh/Data_Train.xlsx
/kaggle/input/flight-fare-prediction-mh/Sample_submission.xlsx
/kaggle/input/flight-fare-prediction-mh/Test_set.xlsx


## Importing Libraries and Datasets

In [194]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [195]:
train_data=pd.read_excel('../input/flight-fare-prediction-mh/Data_Train.xlsx')

In [196]:
train_data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [197]:
train_data['Airline'].value_counts()

Jet Airways                          3849
IndiGo                               2053
Air India                            1752
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: Airline, dtype: int64

In [198]:
train_data['Additional_Info'].value_counts()

No info                         8345
In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
1 Short layover                    1
Red-eye flight                     1
2 Long layover                     1
Name: Additional_Info, dtype: int64

In [199]:
train_data['Source'].value_counts()

Delhi       4537
Kolkata     2871
Banglore    2197
Mumbai       697
Chennai      381
Name: Source, dtype: int64

In [200]:
train_data['Destination'].value_counts()

Cochin       4537
Banglore     2871
Delhi        1265
New Delhi     932
Hyderabad     697
Kolkata       381
Name: Destination, dtype: int64

In [201]:
duration=train_data['Duration'].tolist()

In [202]:
type(duration[0])

str

### Preprocessing the Train Data

In [203]:
# Checking for Null Values
train_data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [204]:
# Dropping Null Values
train_data.dropna(inplace=True)

In [205]:
train_data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
dtype: int64

In [206]:
#Converting duration into duration hour and duration minute
duration_hr=[]
duration_min=[]

for dur in duration:
    if(len(dur.split())<2):
        if dur[-1]=='m':
            dur="0h "+dur
    duration_hr.append((int)(dur.split('h')[0]))
# print(duration_hr)

# print('/n')
for dur in duration:
    if(len(dur.split())==2):
        duration_min.append(dur.split()[1])
    else:
        dur=dur+" 0m"
        duration_min.append(dur.split()[1])
        
duration_min=[(int)(dur.split('m')[0]) for dur in duration_min]
    

In [207]:
print(len(duration))
print(len(duration_hr))
print(len(duration_min))

10683
10683
10683


In [208]:
# Adding Duration_hr and Duration_min in train_data 
train_data["Duration_hr"]=pd.Series(duration_hr)
train_data["Duration_min"]=pd.Series(duration_min)

#Deleting Duration column since it is of no longer use
train_data.drop(['Duration'],axis=1,inplace=True)

In [209]:
train_data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
Duration_hr        0
Duration_min       0
dtype: int64

In [210]:
type((train_data['Duration_min'])[0])

numpy.int64

In [211]:
type((train_data['Duration_hr'])[0])

numpy.int64

In [213]:
#Extracting Hours and Minutes from Arrival Time
train_data['Arrival_hr']=pd.to_datetime(train_data.Arrival_Time).dt.hour
train_data['Arrival_min']=pd.to_datetime(train_data.Arrival_Time).dt.minute

In [214]:
##Extracting Hours and Minutes from Departure Time
train_data['Dep_hr']=pd.to_datetime(train_data.Dep_Time).dt.hour
train_data['Dep_min']=pd.to_datetime(train_data.Dep_Time).dt.minute

In [215]:
#Dropping Arrival_Time and Dep_Time column since they are of no use
train_data.drop(['Arrival_Time'],axis=1,inplace=True)
train_data.drop(['Dep_Time'],axis=1,inplace=True)

In [216]:
train_data

,Airline,Date_of_Journey,Source,Destination,Route,Total_Stops,Additional_Info,Price,Duration_hr,Duration_min,Arrival_hr,Arrival_min,Dep_hr,Dep_min
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,non-stop,No info,3897,2,50,1,10,22,20
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,No info,7662,7,25,13,15,5,50
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,No info,13882,19,0,4,25,9,25
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,1 stop,No info,6218,5,25,23,30,18,5
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,1 stop,No info,13302,4,45,21,35,16,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,non-stop,No info,4107,2,30,22,25,19,55
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,non-stop,No info,4145,2,35,23,20,20,45
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,non-stop,No info,7229,3,0,11,20,8,20
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,non-stop,No info,12648,2,40,14,10,11,30


In [217]:
type((train_data['Date_of_Journey'])[0])

str

In [218]:
#Extracting day,month and year from Date_of_Journey
train_data['Date_of_Journey']=pd.to_datetime(train_data['Date_of_Journey'])
train_data['Date_of_Journey']

0       2019-03-24
1       2019-01-05
2       2019-09-06
3       2019-12-05
4       2019-01-03
           ...    
10678   2019-09-04
10679   2019-04-27
10680   2019-04-27
10681   2019-01-03
10682   2019-09-05
Name: Date_of_Journey, Length: 10682, dtype: datetime64[ns]

In [219]:
train_data['Journey_day']=train_data['Date_of_Journey'].dt.day
train_data['Journey_month']=train_data['Date_of_Journey'].dt.month
train_data['Journey_year']=train_data['Date_of_Journey'].dt.year

In [220]:
train_data

,Airline,Date_of_Journey,Source,Destination,Route,Total_Stops,Additional_Info,Price,Duration_hr,Duration_min,Arrival_hr,Arrival_min,Dep_hr,Dep_min,Journey_day,Journey_month,Journey_year
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,non-stop,No info,3897,2,50,1,10,22,20,24,3,2019
1,Air India,2019-01-05,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,No info,7662,7,25,13,15,5,50,5,1,2019
2,Jet Airways,2019-09-06,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,No info,13882,19,0,4,25,9,25,6,9,2019
3,IndiGo,2019-12-05,Kolkata,Banglore,CCU → NAG → BLR,1 stop,No info,6218,5,25,23,30,18,5,5,12,2019
4,IndiGo,2019-01-03,Banglore,New Delhi,BLR → NAG → DEL,1 stop,No info,13302,4,45,21,35,16,50,3,1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-09-04,Kolkata,Banglore,CCU → BLR,non-stop,No info,4107,2,30,22,25,19,55,4,9,2019
10679,Air India,2019-04-27,Kolkata,Banglore,CCU → BLR,non-stop,No info,4145,2,35,23,20,20,45,27,4,2019
10680,Jet Airways,2019-04-27,Banglore,Delhi,BLR → DEL,non-stop,No info,7229,3,0,11,20,8,20,27,4,2019
10681,Vistara,2019-01-03,Banglore,New Delhi,BLR → DEL,non-stop,No info,12648,2,40,14,10,11,30,3,1,2019


In [221]:
train_data['Journey_year'].value_counts()

2019    10682
Name: Journey_year, dtype: int64

In [222]:
#Dropping Date_of_Journey and Journey_year since they are of no use
train_data.drop(['Date_of_Journey'],axis=1,inplace=True)
train_data.drop(['Journey_year'],axis=1,inplace=True)

In [223]:
train_data

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,Duration_hr,Duration_min,Arrival_hr,Arrival_min,Dep_hr,Dep_min,Journey_day,Journey_month
0,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,No info,3897,2,50,1,10,22,20,24,3
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,No info,7662,7,25,13,15,5,50,5,1
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,No info,13882,19,0,4,25,9,25,6,9
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1 stop,No info,6218,5,25,23,30,18,5,5,12
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,1 stop,No info,13302,4,45,21,35,16,50,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,CCU → BLR,non-stop,No info,4107,2,30,22,25,19,55,4,9
10679,Air India,Kolkata,Banglore,CCU → BLR,non-stop,No info,4145,2,35,23,20,20,45,27,4
10680,Jet Airways,Banglore,Delhi,BLR → DEL,non-stop,No info,7229,3,0,11,20,8,20,27,4
10681,Vistara,Banglore,New Delhi,BLR → DEL,non-stop,No info,12648,2,40,14,10,11,30,3,1


## Categorical Data

In [224]:
from sklearn.preprocessing import OneHotEncoder
train_data=pd.get_dummies(train_data, columns = ['Airline','Source','Destination'])
train_data

,Route,Total_Stops,Additional_Info,Price,Duration_hr,Duration_min,Arrival_hr,Arrival_min,Dep_hr,Dep_min,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,BLR → DEL,non-stop,No info,3897,2,50,1,10,22,20,...,0,0,0,0,0,0,0,0,0,1
1,CCU → IXR → BBI → BLR,2 stops,No info,7662,7,25,13,15,5,50,...,0,0,1,0,1,0,0,0,0,0
2,DEL → LKO → BOM → COK,2 stops,No info,13882,19,0,4,25,9,25,...,0,1,0,0,0,1,0,0,0,0
3,CCU → NAG → BLR,1 stop,No info,6218,5,25,23,30,18,5,...,0,0,1,0,1,0,0,0,0,0
4,BLR → NAG → DEL,1 stop,No info,13302,4,45,21,35,16,50,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,CCU → BLR,non-stop,No info,4107,2,30,22,25,19,55,...,0,0,1,0,1,0,0,0,0,0
10679,CCU → BLR,non-stop,No info,4145,2,35,23,20,20,45,...,0,0,1,0,1,0,0,0,0,0
10680,BLR → DEL,non-stop,No info,7229,3,0,11,20,8,20,...,0,0,0,0,0,0,1,0,0,0
10681,BLR → DEL,non-stop,No info,12648,2,40,14,10,11,30,...,0,0,0,0,0,0,0,0,0,1


In [225]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['Total_Stops']=le.fit_transform(train_data['Total_Stops'])

In [226]:
train_data

,Route,Total_Stops,Additional_Info,Price,Duration_hr,Duration_min,Arrival_hr,Arrival_min,Dep_hr,Dep_min,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,BLR → DEL,4,No info,3897,2,50,1,10,22,20,...,0,0,0,0,0,0,0,0,0,1
1,CCU → IXR → BBI → BLR,1,No info,7662,7,25,13,15,5,50,...,0,0,1,0,1,0,0,0,0,0
2,DEL → LKO → BOM → COK,1,No info,13882,19,0,4,25,9,25,...,0,1,0,0,0,1,0,0,0,0
3,CCU → NAG → BLR,0,No info,6218,5,25,23,30,18,5,...,0,0,1,0,1,0,0,0,0,0
4,BLR → NAG → DEL,0,No info,13302,4,45,21,35,16,50,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,CCU → BLR,4,No info,4107,2,30,22,25,19,55,...,0,0,1,0,1,0,0,0,0,0
10679,CCU → BLR,4,No info,4145,2,35,23,20,20,45,...,0,0,1,0,1,0,0,0,0,0
10680,BLR → DEL,4,No info,7229,3,0,11,20,8,20,...,0,0,0,0,0,0,1,0,0,0
10681,BLR → DEL,4,No info,12648,2,40,14,10,11,30,...,0,0,0,0,0,0,0,0,0,1


In [227]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10682 entries, 0 to 10682
Data columns (total 35 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   Route                                      10682 non-null  object
 1   Total_Stops                                10682 non-null  int64 
 2   Additional_Info                            10682 non-null  object
 3   Price                                      10682 non-null  int64 
 4   Duration_hr                                10682 non-null  int64 
 5   Duration_min                               10682 non-null  int64 
 6   Arrival_hr                                 10682 non-null  int64 
 7   Arrival_min                                10682 non-null  int64 
 8   Dep_hr                                     10682 non-null  int64 
 9   Dep_min                                    10682 non-null  int64 
 10  Journey_day                       

In [228]:
train_data['Additional_Info'].value_counts()

No info                         8344
In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
1 Short layover                    1
Red-eye flight                     1
2 Long layover                     1
Name: Additional_Info, dtype: int64

In [229]:
train_data.drop(['Route'],axis=1,inplace=True)

In [230]:
train_data.drop(['Additional_Info'],axis=1,inplace=True)

In [231]:
train_data

,Total_Stops,Price,Duration_hr,Duration_min,Arrival_hr,Arrival_min,Dep_hr,Dep_min,Journey_day,Journey_month,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,4,3897,2,50,1,10,22,20,24,3,...,0,0,0,0,0,0,0,0,0,1
1,1,7662,7,25,13,15,5,50,5,1,...,0,0,1,0,1,0,0,0,0,0
2,1,13882,19,0,4,25,9,25,6,9,...,0,1,0,0,0,1,0,0,0,0
3,0,6218,5,25,23,30,18,5,5,12,...,0,0,1,0,1,0,0,0,0,0
4,0,13302,4,45,21,35,16,50,3,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,4,4107,2,30,22,25,19,55,4,9,...,0,0,1,0,1,0,0,0,0,0
10679,4,4145,2,35,23,20,20,45,27,4,...,0,0,1,0,1,0,0,0,0,0
10680,4,7229,3,0,11,20,8,20,27,4,...,0,0,0,0,0,0,1,0,0,0
10681,4,12648,2,40,14,10,11,30,3,1,...,0,0,0,0,0,0,0,0,0,1


#### Separating Dependent and Independent Variable

In [232]:
X_train= train_data.loc[:, train_data.columns != 'Price']
y_train= train_data.loc[:,'Price']

In [233]:
X_train

,Total_Stops,Duration_hr,Duration_min,Arrival_hr,Arrival_min,Dep_hr,Dep_min,Journey_day,Journey_month,Airline_Air Asia,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,4,2,50,1,10,22,20,24,3,0,...,0,0,0,0,0,0,0,0,0,1
1,1,7,25,13,15,5,50,5,1,0,...,0,0,1,0,1,0,0,0,0,0
2,1,19,0,4,25,9,25,6,9,0,...,0,1,0,0,0,1,0,0,0,0
3,0,5,25,23,30,18,5,5,12,0,...,0,0,1,0,1,0,0,0,0,0
4,0,4,45,21,35,16,50,3,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,4,2,30,22,25,19,55,4,9,1,...,0,0,1,0,1,0,0,0,0,0
10679,4,2,35,23,20,20,45,27,4,0,...,0,0,1,0,1,0,0,0,0,0
10680,4,3,0,11,20,8,20,27,4,0,...,0,0,0,0,0,0,1,0,0,0
10681,4,2,40,14,10,11,30,3,1,0,...,0,0,0,0,0,0,0,0,0,1


In [234]:
y_train

0         3897
1         7662
2        13882
3         6218
4        13302
         ...  
10678     4107
10679     4145
10680     7229
10681    12648
10682    11753
Name: Price, Length: 10682, dtype: int64

## Preprocessing The Test Data

In [235]:
test_data=pd.read_excel('../input/flight-fare-prediction-mh/Test_set.xlsx')

In [236]:
test_data

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info
...,...,...,...,...,...,...,...,...,...,...
2666,Air India,6/06/2019,Kolkata,Banglore,CCU → DEL → BLR,20:30,20:25 07 Jun,23h 55m,1 stop,No info
2667,IndiGo,27/03/2019,Kolkata,Banglore,CCU → BLR,14:20,16:55,2h 35m,non-stop,No info
2668,Jet Airways,6/03/2019,Delhi,Cochin,DEL → BOM → COK,21:50,04:25 07 Mar,6h 35m,1 stop,No info
2669,Air India,6/03/2019,Delhi,Cochin,DEL → BOM → COK,04:00,19:15,15h 15m,1 stop,No info


In [237]:
test_data.drop(['Route'],axis=1,inplace=True)
test_data.drop(['Additional_Info'],axis=1,inplace=True)

In [238]:
test_data.isna().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
dtype: int64

In [239]:
duration_test=test_data['Duration'].tolist()

duration_hr=[]
duration_min=[]

for dur in duration_test:
    if(len(dur.split())<2):
        if dur[-1]=='m':
            dur="0h "+dur
    duration_hr.append((int)(dur.split('h')[0]))
# print(duration_hr)

# print('/n')
for dur in duration_test:
    if(len(dur.split())==2):
        duration_min.append(dur.split()[1])
    else:
        dur=dur+" 0m"
        duration_min.append(dur.split()[1])
        
duration_min=[(int)(dur.split('m')[0]) for dur in duration_min]


test_data["Duration_hr"]=pd.Series(duration_hr)
test_data["Duration_min"]=pd.Series(duration_min)

test_data.drop(['Duration'],axis=1,inplace=True)

In [240]:
test_data['Arrival_hr']=pd.to_datetime(test_data.Arrival_Time).dt.hour
test_data['Arrival_min']=pd.to_datetime(test_data.Arrival_Time).dt.minute


test_data['Dep_hr']=pd.to_datetime(test_data.Dep_Time).dt.hour
test_data['Dep_min']=pd.to_datetime(test_data.Dep_Time).dt.minute


test_data.drop(['Arrival_Time'],axis=1,inplace=True)
test_data.drop(['Dep_Time'],axis=1,inplace=True)

In [241]:
test_data['Date_of_Journey']=pd.to_datetime(test_data['Date_of_Journey'])

test_data['Journey_day']=test_data['Date_of_Journey'].dt.day
test_data['Journey_month']=test_data['Date_of_Journey'].dt.month

test_data.drop(['Date_of_Journey'],axis=1,inplace=True)

In [242]:
test_data

,Airline,Source,Destination,Total_Stops,Duration_hr,Duration_min,Arrival_hr,Arrival_min,Dep_hr,Dep_min,Journey_day,Journey_month
0,Jet Airways,Delhi,Cochin,1 stop,10,55,4,25,17,30,6,6
1,IndiGo,Kolkata,Banglore,1 stop,4,0,10,20,6,20,5,12
2,Jet Airways,Delhi,Cochin,1 stop,23,45,19,0,19,15,21,5
3,Multiple carriers,Delhi,Cochin,1 stop,13,0,21,0,8,0,21,5
4,Air Asia,Banglore,Delhi,non-stop,2,50,2,45,23,55,24,6
...,...,...,...,...,...,...,...,...,...,...,...,...
2666,Air India,Kolkata,Banglore,1 stop,23,55,20,25,20,30,6,6
2667,IndiGo,Kolkata,Banglore,non-stop,2,35,16,55,14,20,27,3
2668,Jet Airways,Delhi,Cochin,1 stop,6,35,4,25,21,50,3,6
2669,Air India,Delhi,Cochin,1 stop,15,15,19,15,4,0,3,6


##### Handling Categorical Data

In [243]:
# One Hot Encoding
from sklearn.preprocessing import OneHotEncoder
test_data=pd.get_dummies(test_data, columns = ['Airline','Source','Destination'])


#Label Encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
test_data['Total_Stops']=le.fit_transform(test_data['Total_Stops'])

test_data

,Total_Stops,Duration_hr,Duration_min,Arrival_hr,Arrival_min,Dep_hr,Dep_min,Journey_day,Journey_month,Airline_Air Asia,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,0,10,55,4,25,17,30,6,6,0,...,0,1,0,0,0,1,0,0,0,0
1,0,4,0,10,20,6,20,5,12,0,...,0,0,1,0,1,0,0,0,0,0
2,0,23,45,19,0,19,15,21,5,0,...,0,1,0,0,0,1,0,0,0,0
3,0,13,0,21,0,8,0,21,5,0,...,0,1,0,0,0,1,0,0,0,0
4,4,2,50,2,45,23,55,24,6,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,0,23,55,20,25,20,30,6,6,0,...,0,0,1,0,1,0,0,0,0,0
2667,4,2,35,16,55,14,20,27,3,0,...,0,0,1,0,1,0,0,0,0,0
2668,0,6,35,4,25,21,50,3,6,0,...,0,1,0,0,0,1,0,0,0,0
2669,0,15,15,19,15,4,0,3,6,0,...,0,1,0,0,0,1,0,0,0,0
